In [ ]:
# %%capture
%pip install crewai | tail -n 1
%pip install crewai-tools | tail -n 1
%pip install langchain-community | tail -n 1
%pip install langchain-huggingface | tail -n 1
%pip install sentence-transformers | tail -n 1

In [1]:
%%capture

from crewai import Agent, Task, Crew, Process
from crewai import LLM
from crewai_tools import PDFSearchTool, SerperDevTool

In [2]:
llm = LLM(
    model="watsonx/ibm/granite-3-3-8b-instruct",
    base_url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
)

In [3]:
import os
os.environ['SERPER_API_KEY'] = '********'

In [4]:
web_search_tool = SerperDevTool()

In [5]:
import warnings
warnings.filterwarnings('ignore') #Keeps Jupyter Notebook clean (not part of functionality)

pdf_search_tool = PDFSearchTool(
    pdf="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7vgNfis17dQfjHAiIKkBOg/The-Daily-Dish-FAQ.pdf",
    config=dict(
        embedder=dict(
            provider="huggingface",
            config=dict(
                model="sentence-transformers/all-MiniLM-L6-v2"
            )
        )
    )
)

In [6]:
agent_centric_agent = Agent(
    role="The Daily Dish Inquiry Specialist",
    goal="""Accurately answer customer questions about The Daily Dish restaurant. 
    You must decide whether to use the restaurant's FAQ PDF or a web search to find the best answer.""",
    backstory="""You are an AI assistant for 'The Daily Dish'.
    You have access to two tools: one for searching the restaurant's FAQ document and another for searching the web.
    Your job is to analyze the user's question and choose the most appropriate tool to find the information needed to provide a helpful response.""",
    tools=[pdf_search_tool, web_search_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [7]:
agent_centric_task = Task(
    description="Answer the following customer query: '{customer_query}'. "
                "Analyze the question and use the tools at your disposal (PDF search or web search) to find the most relevant information. "
                "Synthesize the findings into a clear and friendly response.",
    expected_output="A comprehensive and well-formatted answer to the customer's query.",
    agent=agent_centric_agent
)

In [8]:
agent_centric_crew = Crew(
    agents=[agent_centric_agent],
    tasks=[agent_centric_task],
    process=Process.sequential,
    verbose=False
)

In [9]:
# Download the FAQ document for the tool to use
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7vgNfis17dQfjHAiIKkBOg/The-Daily-Dish-FAQ.pdf

--2025-09-08 00:32:50--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7vgNfis17dQfjHAiIKkBOg/The-Daily-Dish-FAQ.pdf
169.63.118.104ourses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53993 (53K) [application/pdf]
Saving to: ‘The-Daily-Dish-FAQ.pdf.1’

The-Daily-Dish-FAQ. 100%[===================>]  52.73K  --.-KB/s    in 0.002s  

2025-09-08 00:32:50 (33.7 MB/s) - ‘The-Daily-Dish-FAQ.pdf.1’ saved [53993/53993]



In [ ]:
print("\nWelcome to The Daily Dish Chatbot!")
print("What would you like to know? (Type 'exit' to quit)")

while True: 
    user_input = input("\nYour question: ").lower()
    if user_input == 'exit':
        print("Thank you for chatting. Have a great day!")
        break
    
    if not user_input:
        print("Please type a question.")
        continue

    try:
        result_agent_centric = agent_centric_crew.kickoff(inputs={'customer_query': user_input})
        print("\n--- The Daily Dish Assistant ---")
        print(result_agent_centric)
        print("--------------------------------")
    except Exception as e:
        print(f"An error occurred: {e}")


Welcome to The Daily Dish Chatbot!
What would you like to know? (Type 'exit' to quit)



Your question:  phone number?


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Task: Answer the following customer query: 'phone number?'. Analyze the question and use the tools at your     │
│  disposal (PDF search or web search) to find the most relevant information. Synthesize the findings into a      │
│  clear and friendly response.                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Thought: Thought: The customer is asking for the phone number of The Daily Dish restaurant. The FAQ document   │
│  is the most appropriate place to find this information.                                                        │
│                                                                                                                 │
│  Using Tool: Search a PDF's content                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"phone number?\"}"                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Thought: The customer query requires information about restaurants located near The Daily Dish. This           │
│  information is not likely to be available in the restaurant's FAQ PDF. Therefore, a web search would be more   │
│  appropriate to find restaurants in the vicinity of The Daily Dish.                                             │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"restaurants near The Daily Dish\"}"                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'restaurants near The Daily Dish', 'type': 'search', 'num': 10, 'engine':           │
│  'google'}, 'organic': [{'title': 'Home - The Daily Dish Restaurant', 'link':                                   │
│  'https://thedailydishrestaurant.com/', 'snippet': 'Discover The Daily Dish, a top-rated restaurant and bar     │
│  located at 8301 Grubb Road, Silver Spring, MD 20910. Nestled in Montgomery County, Maryland, ...',             │
│  'position': 1, 'sitelinks': [{'title': 'About', 'link': 'https://thedailydishrestaurant.com/about/'},          │
│  {'title': 'Dinner Menu', 'link': 'https://thedailydishrestaurant.com/dinner/'}, {'title': 'Lunch Menu',        │
│  'link': 'https://thedailydishrestaurant.com/lunch/'}, {'title': 'Saturday and Sunday Brunch...', 'link':       │
│  'https://thedailydishrestaurant.com/brunch/'}]}, {'title': 'THE BEST 10 RESTAURANTS near S MAJOR DR,           │
│  BEAUMONT, TX', 'link': 'https://www.yelp.com/search?cflt=restaurants&find_loc=S+Major+Dr%2C+Beaumont%2C+TX',   │
│  'snippet': "The Best 10 Restaurants near S Major Dr, Beaumont, TX · 1. Wicked Casian · 2. Riverside Grille ·   │
│  3. JW's Patio · 4. Floyds Seafood - Beaumont · 5. Hole in the Wall.", 'position': 2}, {'title': 'What are      │
│  some of your favorite restaurants in SETX? : r/Beaumont', 'link':                                              │
│  'https://www.reddit.com/r/Beaumont/comments/tzac0c/what_are_some_of_your_favorite_restaurants_in_setx/',       │
│  'snippet': "Here is my list in no particular order :) I've heard many people say Sweet Basil, Crown Pizza,     │
│  Willy Burger & Daddio's are all great as well.", 'position': 3}, {'title': 'Daily Dish - a cooking and dining  │
│  blog', 'link': 'https://dailydish.com/', 'snippet': 'Where can I find little bursts of joy these days? For     │
│  me, it is in new culinary experiences, i.e. going to new restaurants. Pre-pandemic, I could look to lunch      │
│  ...', 'position': 4}, {'title': 'Top 11 Places to Eat in Beaumont, Texas | Foodie Edition', 'link':            │
│  'https://www.beaumontcvb.com/restaurants/favorite-foodie-restaurants-in-beaumont/', 'snippet': 'Savor Cajun,   │
│  Tex-Mex & BBQ at 11 must-try spots around Beaumont recommended by t...                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here are some restaurants near The Daily Dish:                                                                 │
│                                                                                                                 │
│  1. **Wicked Casian** (Yelp) - This restaurant, located in Beaumont, TX, is highly-rated and offers a unique    │
│  dining experience.                                                                                             │
│                                                                                                                 │
│  2. **Riverside Grille** (Yelp) - Another top-rated restaurant in Beaumont, known for its grill and bar         │
│  offerings.                                                                                                     │
│                                                                                                                 │
│  3. **JW's Patio** (Yelp) - A popular favorite in Beaumont, JW's Patio offers a cozy outdoor dining             │
│  atmosphere.                                                                                                    │
│                                                                                                                 │
│  4. **Floyds Seafood - Beaumont** (Yelp) - A seafood-focused restaurant that has received high reviews from     │
│  customers.                                                                                                     │
│                                                                                                                 │
│  5. **Hole in the Wall** (Yelp) - Though it seems to be a small restaurant according to its name, it's favored  │
│  by the local community.                                                                                        │
│                                                                                                                 │
│  6. [Restaurant Hearsay](https.www.opentable.com/nearby/restaurants-near-me-beaumont-tx) - Available on         │
│  OpenTable, this restaurant in Beaumont offers a diverse menu with various appetizers, entrees, and desserts.   │
│                                                                                                                 │
│  7. [Sushi Masa Beaumont](https.www.opentable.com/nearby/restaurants-near-me-beaumont-tx) - Another restaurant  │
│  featured on OpenTable, Sushi Masa Beaumont satisfies seafood and sushi lovers.                                 │
│                                                                                                                 │
│  8. [The Texas Star Bar & Grill](https.www.opentable.com/nearby/restaurants-near-me-beaumont-tx) - This         │
│  Beaumont establishment offers a casual dining setting with a bar.                                              │
│                                                                                                                 │
│  9. [Rosewater Grill & Tavern - Delta Downs](https.www.opentable.com/nearby/restaurants-near-me-beaumont-tx) -  │
│  Featured on OpenTable, Rosewater Grill offers a diverse menu with an extensive wine list.                      │
│                                                                                                                 │
│  10. [10 Best Restaurants in Beaumont (Updated Septembe


--- The Daily Dish Assistant ---
Here are some restaurants near The Daily Dish:

1. **Wicked Casian** (Yelp) - This restaurant, located in Beaumont, TX, is highly-rated and offers a unique dining experience.

2. **Riverside Grille** (Yelp) - Another top-rated restaurant in Beaumont, known for its grill and bar offerings.

3. **JW's Patio** (Yelp) - A popular favorite in Beaumont, JW's Patio offers a cozy outdoor dining atmosphere.

4. **Floyds Seafood - Beaumont** (Yelp) - A seafood-focused restaurant that has received high reviews from customers.

5. **Hole in the Wall** (Yelp) - Though it seems to be a small restaurant according to its name, it's favored by the local community.

6. [Restaurant Hearsay](https.www.opentable.com/nearby/restaurants-near-me-beaumont-tx) - Available on OpenTable, this restaurant in Beaumont offers a diverse menu with various appetizers, entrees, and desserts.

7. [Sushi Masa Beaumont](https.www.opentable.com/nearby/restaurants-near-me-beaumont-tx) - Anoth